In [1]:
print("Hello Parth!")

Hello Parth!


In [75]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

def load_pdf(pdf_path):
    loader = PyPDFLoader(file_path=pdf_path)
    documents = loader.lazy_load()
    docs = []
    for doc in documents:
        docs.append(doc)
    return docs

extracted_data = load_pdf("StandardVeterinaryTreatment.pdf")

In [98]:
print(extracted_data[100].page_content)

83
STANDARD VETERINARY TREATMENT GUIDELINES FOR LIVESTOCK AND POULTRY
enzootic bovine leukaemia (EBL), is a contagious 
disease in cattle caused by the bovine leukaemia 
virus (BLV). It belongs to the genus Deltaretrovirus 
in family Retroviridae. Retrovirus integrates into 
the host’s DNA and can lead to the development 
of lymphosarcoma, a type of cancer affecting the 
lymphatic system.
Transmission
Bovine leukaemia virus (BLV) primarily transmits 
through infected blood or body fluids. Transmission 
can occur during procedures like dehorning, 
castration, and vaccination if hygiene is inadequate. 
Biting insects and flies can also facilitate virus 
transmission between cattle. Infected cows may 
transmit BLV to their calves during pregnancy and 
calves can acquire the virus through colostrum 
or milk consumption. The virus can spread via 
contaminated needles, surgical tools, and other 
equipment used for injections and blood sampling, 
as well as through contaminated gloves during 

In [78]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [79]:
text_chunks = text_split(extracted_data)
print(f"Length of text chunks : {len(text_chunks)}")

Length of text chunks : 5759


In [81]:
print(text_chunks[5500].page_content)

and cytochrome CYP3A4, so it can modify the 
concentration of majority of drugs.
Capsaicin (Chili pepper) is vasodilator, analgesic, 
antibacterial, antioxidant, anticancer, and 
antidiabetic. It is contraindicated with aspirin, 
clopidogrel, diclofenac, naproxen, ibuprofen, oral


In [82]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Download Embedding Model
def hugging_face_embedding():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", show_progress=True)
    return embedding

In [83]:
embedding = hugging_face_embedding()

In [11]:
embedding

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=True)

In [84]:
result = embedding.embed_query("Parth Dobariya")
print(f"Length of Vector : {len(result)}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 65.85it/s]

Length of Vector : 384


In [85]:
from langchain_chroma import Chroma

persist_directory = "db"

chroma_vectordb = Chroma.from_documents(documents=text_chunks, embedding=embedding, persist_directory=persist_directory)

Batches: 100%|██████████| 180/180 [01:44<00:00,  1.73it/s]


In [86]:
chroma_vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [87]:
chroma_vectordb

In [88]:
# Make retriever
retriever = chroma_vectordb.as_retriever()

In [89]:
docs = retriever.invoke("Definition of Camel contagious ecthyma")

Batches: 100%|██████████| 1/1 [00:00<00:00, 58.85it/s]


In [90]:
for i in docs:
    print(i.page_content)
    print("="*50)

The camel contagious ecthyma (CCE) – also 
known as contagious pustular dermatitis (CPD), 
sore mouth, or orf – is  a highly contagious viral 
disease, mostly of young camels, that contributes 
to calf debility and, occasionally, calf loss. CCE is
The camel contagious ecthyma (CCE) – also 
known as contagious pustular dermatitis (CPD), 
sore mouth, or orf – is  a highly contagious viral 
disease, mostly of young camels, that contributes 
to calf debility and, occasionally, calf loss. CCE is
oral swabs are taken. If gastrointestinal symptoms 
are present, then a rectal swab should be collected. 
Blood samples are required for immunological and 
molecular diagnostic tests.
7.2.1.2. Camel contagious ecthyma 
Definition and causative agent 
The camel contagious ecthyma (CCE) – also
oral swabs are taken. If gastrointestinal symptoms 
are present, then a rectal swab should be collected. 
Blood samples are required for immunological and 
molecular diagnostic tests.
7.2.1.2. Camel contagious e

In [91]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAI

load_dotenv()
GOOGLE_GEMINI_API = os.getenv("GOOGLE_GEMINI_API")

llm = GoogleGenerativeAI(model="gemini-pro", api_key=GOOGLE_GEMINI_API)

In [92]:
from langchain.prompts import ChatPromptTemplate

system_prompt = (
    "Use the following pieces of information to answer to user's question."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    
    "Context: {context}"
)

PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [93]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = chroma_vectordb.as_retriever(search_kwargs={'k': 2})
question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=PROMPT)
qa_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=question_answer_chain)

In [94]:
query = "Definition of Camel contagious ecthyma"
response = qa_chain.invoke({"input": query})

Batches: 100%|██████████| 1/1 [00:00<00:00, 56.68it/s]


In [95]:
response["answer"]

'Camel contagious ecthyma (CCE) is a highly contagious viral disease, mostly of young camels, that contributes to calf debility and, occasionally, calf loss.'

In [96]:
while True:
    user_query = input(f"Input Prompt").lower()
    if user_query == "exit" or user_query == "":
        break
    else:
        response = qa_chain.invoke({"input": user_query})
        print(f"Response : {response['answer']}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 76.00it/s]


Response : Camel contagious ecthyma (CCE) is a highly contagious viral disease, mostly of young camels, that contributes to calf debility and, occasionally, calf loss.
